# Image Quality
- Brisque - https://pypi.org/project/brisque/
- image-quality 1.2.7 - https://pypi.org/project/image-quality/
- NIMA - https://github.com/yunxiaoshi/Neural-IMage-Assessment

For assessing image quality we have to consider two aspects. Technical and aesthetic quality of an image. Our usage involves no reference for quality assessment.We are going to look into how fast and sensitive the image quality evaluator is.

Firstly we are going to focus on technical quality. For technical quality we can choose from two approaches, one that is algorithmic and focused on scene statistics and the second one, which is using CNN trained on TID2013 dataset.

For algorithmic approach we tested two implementations of the same algorithm called BRISQUE (Blind/referenceless image spatial quality evaluator). First implementation is library created by Rehan Guha (https://pypi.org/project/brisque/). 

In [126]:
import time,os
from brisque import BRISQUE
from skimage import io

In [127]:
# We create list of images that we use for testing

image_path = '/home/lukas/Bakalářka/photo_culling/images/testing'
img_list = []  # list of image file names to process
for path in os.scandir(image_path):
    if path.is_file():
        if path.name.endswith(".jpg"):
            img_list += [path.name]

In [128]:
obj = BRISQUE(url=False)
results_BRISQUE = []

tic = time.perf_counter()
for img in img_list:
    x = io.imread(os.path.join(image_path,img))
    results_BRISQUE.append(obj.score(x))
toc = time.perf_counter()
print(f"TIME - {toc - tic:0.2f} s")
print(img_list)
print(results_BRISQUE)

TIME - 44.76 s
['clear.jpg', 'GaussBlur.jpg', 'rotated.jpg', 'inverted.jpg', 'hue_shift.jpg', 'contrast.jpg']
[63.439140291231155, 96.60843292281524, 63.230106082455876, 63.30271773240301, 59.23486617151738, 71.03791634023887]


From this first test we can see that for this algorithm rotating and inverting has little to no effect. For shifted hue and increased contrast we can observe
small change in quality score. And for Gaussian blur we can see the biggest quality score drop. This is expected as the technical quality should only be measured
by pixels relation to its surroundings.

Now we can try testing the second implementation of BRISQUE from image-quality library made by Ricardo Ocampo. In this implementation we need to open the images
with Pillow image library function. This is slight downside.

In [129]:
import PIL.Image
import imquality.brisque

In [130]:
results_BRISQUE = []

tic = time.perf_counter()
for img in img_list:
    x = imquality.brisque.score(PIL.Image.open(os.path.join(image_path,img)))
    results_BRISQUE.append(x)
toc = time.perf_counter()
print(f"TIME - {toc - tic:0.2f} s")
print(img_list)
print(results_BRISQUE)

TIME - 242.50 s
['clear.jpg', 'GaussBlur.jpg', 'rotated.jpg', 'inverted.jpg', 'hue_shift.jpg', 'contrast.jpg']
[63.76477488661598, 98.15799411945116, 63.57080624559066, 63.595836343125, 59.454852915252076, 71.33528411947336]


From the second test we can observe the same as in the first one. The most significant score change is in the case of blurring the image. The quality scores
for both implementations are nearly identical.

Both of these solutions are very easy-to-use, give us single score number representing the technical quality and are linear in terms of computing time. That said,
the first implementation is around 5 times faster and doesn't require us to open images beforehand. From these factors we can already see that the first
implementation is superior of those two.

Now we can go ahead and test CNN approach to technical quality assessment.